In [1]:
import numpy

In [3]:
a = numpy.zeros([100000, 500])

In [6]:
b = [a.copy()] * 10

In [2]:
file_path = 'data/glove300.txt'

In [ ]:
model_en = gensim.models.KeyedVectors.load_word2vec_format(file_path)